In [10]:
import triton
import triton.language as tl
import torch
from einops import rearrange
from triton import cdiv

In [ ]:
@triton.jit  # 修饰器，告诉python解释器把它编译成 GPU 机器码，并在 GPU 上运行。
def weighted_sum_fwd(
    x_ptr, weight_ptr,      # 输入指针
    output_ptr,             # 输出指针
    x_stride_row, x_stride_dim,      # 步长告诉我们如何在张量的每个轴上移动一个元素
    weight_stride_dim,
    output_stride_row,
    ROWS, D,
    ROWS_TILE_SIZE: tl.constexpr, D_TILE_SIZE: tl.constexpr,   # 分块形状必须在编译时已知
):
    # 每个实例将计算 x 的一个行分块的加权和
    # 'tl.program_id' 给出当前正在运行的 “程序实例 (Program Instance)” 的 ID
    # 当前这个程序实例负责处理输入矩阵的第几个行分块 (Row Tile)
    row_tile_idx = tl.program_id(0)    # 0 指的网格的第0维 (即x轴) 


    # 块指针 (Block pointers) 为我们提供一种从内存的 ND (N维) 区域中进行选择
    # 并移动我们选择区域的方法.
    # 块指针必须知道的参数:
    # - 指向张量第一个元素的指针
    # - 张量的整体形状, 以处理越界访问
    # - 每个维度的步长，以正确使用内存布局
    # - 其实块的 ND 坐标, 即"偏移量 (offset) "
    # - (block shape) 当前这个 Kernel 实例一次性要加载到芯片缓存（SRAM）里处理的一小块区域的大小
    # - 内存中维度的顺序，从主序到次序

    x_block_ptr = tl.make_block_ptr(
        x_ptr,
        shape=(ROWS, D,),
        strides=(x_stride_row, x_stride_dim),
        offsets=(row_tile_idx * ROWS_TILE_SIZE, 0),
        block_shape=(ROWS_TILE_SIZE, D_TILE_SIZE), 
        order=(1,0), # order 参数要求传入一个元组，代表“按步长（Stride）从小到大排序的维度索引”
    )

    weight_block_ptr = tl.make_block_ptr(
        weight_ptr,
        shape=(D,),
        strides=(weight_stride_dim,),
        offsets=(0,),
        block_shape=(D_TILE_SIZE,),
        order=(0,),
    )

    output_block_ptr = tl.make_block_ptr(
        output_ptr,
        shape=(ROWS,),
        strides=(output_stride_row,),
        offsets=(row_tile_idx * ROWS_TILE_SIZE,),  # 输出时候的偏移量 确定当前线程output应该放在哪
        block_shape=(ROWS_TILE_SIZE,),
        order=(0,),
    )

    # 初始化一个缓冲区用于写入
    output = tl.zeros((ROWS_TILE_SIZE,), dtype=tl.float32)

    for i in range(tl.cdiv(D, D_TILE_SIZE)):
        # 加载当前的块指针
        # 由于 ROWS_TILE_SIZE 可能无法整除 ROWS, 且 D_TILE_SIZE 可能无法整除 D
        # 因此需要对两个维度进行边界检查
        row = tl.load(x_block_ptr, boundary_check=(0, 1), padding_option="zero") # (ROWS_TILE_SIZE, D_TILE_SIZE)
        weight = tl.load(weight_block_ptr, boundary_check=(0,), padding_option="zero") # (D_TILE_SIZE,)

        # 计算行的加权和
        output += tl.sum(row * weight[None, :], axis=1)

        # 将指针移动到下一个分块
        # 这些都是 (rows, columns) 坐标的增量
        x_block_ptr = x_block_ptr.advance((0, D_TILE_SIZE))  # # 在最后一个维度移动 D_TILE_SIZE
        weight_block_ptr = weight_block_ptr.advance((D_TILE_SIZE,))

    # 将输出写入输出块指针 (每行一个标量)
    tl.store(output_block_ptr, output, boundary_check=(0,))

In [17]:
@triton.jit
def weighted_sum_backward(
    x_ptr, weight_ptr,
    grad_output_ptr,  # 梯度输入
    grad_x_ptr, partial_grad_weight_ptr,  # 梯度输出
    stride_xr, stride_xd,
    stride_wd,
    stride_gr,
    stride_gxr, stride_gxd,
    stride_gwb, stride_gwd,
    NUM_ROWS, D,
    ROWS_TILE_SIZE: tl.constexpr, D_TILE_SIZE: tl.constexpr,
):
    row_tile_idx = tl.program_id(0)
    n_row_tiles = tl.num_programs(0)

    # 输入块指针定义
    grad_output_block_ptr = tl.make_block_ptr(
        grad_output_ptr,
        shape=(NUM_ROWS,), strides=(stride_gr),
        offsets=(row_tile_idx * ROWS_TILE_SIZE,),
        block_shape=(ROWS_TILE_SIZE,),
        order=(0,),
    )

    x_block_ptr = tl.make_block_ptr(
        x_ptr,
        shape=(NUM_ROWS, D, ), strides=(stride_xr, stride_xd),
        offsets=(row_tile_idx * ROWS_TILE_SIZE, 0),
        block_shape=(ROWS_TILE_SIZE, D_TILE_SIZE),
        order=(1, 0),
    )

    weight_block_ptr = tl.make_block_ptr(
        weight_ptr,
        shape=(D,), strides=(stride_wd,),
        offsets=(0,), block_shape=(D_TILE_SIZE,),
        order=(0,),
    )

    grad_x_block_ptr = tl.make_block_ptr(
        grad_x_ptr,
        shape=(NUM_ROWS, D,), strides=(stride_gxr, stride_gxd),
        offsets=(row_tile_idx * ROWS_TILE_SIZE, 0),
        block_size=(ROWS_TILE_SIZE, D_TILE_SIZE),
        order=(1, 0),
    )
    partial_grad_weight_block_ptr = tl.make_block_ptr(
        partial_grad_weight_ptr,
        shape=(n_row_tiles, D,), strides=(stride_gwb, stride_gwd),
        offsets=(row_tile_idx, 0),
        block_shape=(1, D_TILE_SIZE),
        order=(1, 0),
    )

    for i in range(tl.cdiv(D, D_TILE_SIZE)):
        grad_output = tl.load(grad_output_block_ptr, boundary_check=(0,), padding_option="zero")

        # 计算 grad_x 的外积
        weight = tl.load(weight_block_ptr, boundary_check=(0,), padding_option="zero")
        grad_x_row = grad_output[:, None] * weight[None, :]
        tl.store(grad_x_block_ptr, grad_x_row, boundary_check=(0, 1))

        # 为 grad_wight 结果尽可能多行并行
        row = tl.load(x_block_ptr, boundary_check=(0, 1), padding_option="zero")
        grad_weight_row = tl.sum(row * grad_output[:, None], axis=0, keep_dims=True)
        tl.store(partial_grad_weight_block_ptr, grad_weight_row, boundary_check=(1,))

        # 沿着 D 移动指针到下一个分块
        x_block_ptr = x_block_ptr.advance((0, D_TILE_SIZE))
        weight_block_ptr = weight_block_ptr.advance((D_TILE_SIZE,))
        partial_grad_weight_block_ptr = partial_grad_weight_block_ptr.advance((0, D_TILE_SIZE))
        grad_x_block_ptr = grad_x_block_ptr.advance((0, D_TILE_SIZE))
        

In [4]:
x.stride(0)

20

In [ ]:
class WeightedSumFunc(torch.autograd.Function):
    """
    torch.autograd.Function: 基类, 手动定义一个算子的前向传播 (Forward) 和反向传播 (Backward) 逻辑
    一个自定义的 Function 必须继承这个类，并实现两个静态方法 (@staticmethod) forward 和 backward
    """
    @staticmethod
    def forward(ctx, x, weight):
        D, output_dims = x.shape[-1], x.shape[:-1]

        # 将输入张量重塑(Reshape) 为 2D
        input_shape = x.shape
        x = rearrange(x, "... d -> (...) d")
        # 缓存 x 和 weight 以便于反向传播中使用，届时我们只会接收到关于输出张量的梯度，而需要计算关于 x 和 weight 的梯度
        ctx.save_for_backward(x, weight)

        assert len(weight.shape) == 1 and weight.shape[0] == D, "Dimension mismatch"
        assert x.is_cuda and weight.is_cuda, "Excepted CUDA tensors"
        assert x.is_contiguous(), "Our pointer arithmetic will assume contiguous x"

        ctx.D_TILE_SIZE = triton.next_power_of_2(D) // 16  # 列块大小, 循环次数
        ctx.ROWS_TILE_SIZE = 16  # 每个线程块一次处理 16 行批次元素,行块大小, 假设有N=100行, 则需要启动 100 / 16 = 7 个线程块并行进行
        ctx.input_shape = input_shape
        
        y = torch.empty(output_dims, device=x.device)

        # 在我们的 1D 网格中启动 n 个实例来运行我们的代码
        n_rows = y.numel()  # 总共有多少行,即总任务数
        """
        当我们使用 weighted_sum_fwd[(cdiv(n_rows, ctx.ROWS_TILE_SIZE),)] 调用 Triton 内核时，
        我们通过传递元组 (cdiv(n_rows, ctx.ROWS_TILE_SIZE),) 定义了一个所谓的“启动网格 (launch grid)”（线程块的网格）。
        然后，我们可以在内核中使用 tl.program_id(0) 访问线程块的索引。
        """
        weighted_sum_fwd[(cdiv(n_rows, ctx.ROWS_TILE_SIZE),)](  # n_rows / ctx.ROWS_TILE_SIZE 个线程并行进行
            x, weight,
            y,
            x.stride(0), x.stride(1),
            weight.stride(0),
            y.stride(0),
            ROWS=n_rows, D=D,
            ROWS_TILE_SIZE=ctx.ROWS_TILE_SIZE, D_TILE_SIZE=ctx.D_TILE_SIZE,
        )
        return y.view(input_shape[:-1])
    
    @staticmethod
    def backward(ctx, grad_out):
        x, weight = ctx.saved_tensors
        ROWS_TILE_SIZE, D_TILE_SIZE = ctx.ROWS_TILE_SIZE, ctx.D_TILE_SIZE  
        n_rows, D = x.shape

        # 让每个线程块先写入一部分缓冲区，然后在该缓冲区上进行归约以获得最终梯度
        partial_grad_weight = torch.empty((cdiv(n_rows, ROWS_TILE_SIZE), D), device=x.device, dtype=x.dtype)
        grad_x = torch.empty_like(x)

        weighted_sum_backward[(cdiv(n_rows, ROWS_TILE_SIZE),)](
            x, weight,
            grad_out,
            grad_x, partial_grad_weight,
            x.stride(0), x.stride(1),
            weight.stride(0),
            grad_out.stride(0),
            grad_x.stride(0), grad_x.stride(1),
            partial_grad_weight.stride(0), partial_grad_weight.stride(1),
            NUM_ROWS=n_rows, D=D,
            ROWS_TILE_SIZE=ROWS_TILE_SIZE, D_TILE_SIZE=D_TILE_SIZE,
        )

        grad_weight = partial_grad_weight.sum(axis=0)
        return grad_x, grad_weight

In [ ]:
"""
必须通过 .apply 才能连接自动求导
WeightedSumFunc 是一个继承自 torch.autograd.Function 的类。

你不能直接实例化它：model = WeightedSumFunc() ❌ (这是错的)

你也不能直接调用 forward：y = WeightedSumFunc.forward(ctx, x, w) ❌ (这是错的)

.apply 是 PyTorch 内部的一个魔法方法。当你调用 WeightedSumFunc.apply(x, w) 时，PyTorch 会在后台做以下事情：

创建 ctx (Context) 对象。作用是在 前向传播 (Forward) 和 反向传播 (Backward)。如 ctx.save_for_backward(x, weight)

连接前向传播和反向传播的节点。

确保梯度能够流过这个操作。

所以，你每次使用这个算子，都必须写 WeightedSumFunc.apply(...)。
"""
f_weightedsum = WeightedSumFunc.apply  # 定义别名

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
x = torch.randn(100, 64, device=device, requires_grad=True)
weight = torch.randn(64, device=device, requires_grad=True)

In [20]:
# 3. 运行你的 Triton 算子
y = f_weightedsum(x, weight)

In [21]:
y

tensor([ 11.9563,  -8.3941,  14.7706,   2.6851,   0.5113,  -5.6934,  -7.7887,
        -16.7739,   0.2786,   1.6083,  -9.1864,  -3.1714,  -4.2149,  -4.5889,
         -1.2386,   5.8556, -10.2032,   9.6943,   9.2802,  -5.1951,   0.4868,
         -4.7471,  10.7786,   9.2061,  -2.8172,  10.0051, -11.6117,   9.3835,
          6.9188,  18.2391,  -7.1230,   4.0649,   4.8202,   0.4916,  11.2586,
          5.6963,  11.7063,   6.4597,   0.4835,   9.6063,   1.0958,   1.1348,
        -10.2260,  -9.7152,  -0.5819,  -1.5178,  -4.1269,  17.4520,   5.8099,
         -8.6497,  -6.9691,   5.1863,   1.3750,  14.0855,  -2.0366,   6.6039,
          9.8486, -16.9976,  20.7936,   8.6135,   8.6118,  14.4096, -11.0249,
          5.9540,  -1.7794,  -5.2389,  -9.9471,   4.4918,   4.3935,   7.7661,
         -4.2371,   2.1426,   5.7930,   9.2507,  15.6980,   6.6524,   4.1412,
         -4.8109,  -1.9191,   3.4117,  -1.3624,  10.2873,  -1.5904,   1.0420,
          4.0504,  -4.3510,  10.1120,  -1.0156,  -1.8766,  -5.03